In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, utils, losses, initializers, metrics, callbacks, preprocessing
from sklearn.model_selection import train_test_split

In [12]:
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = 'checkpoints/model.keras'
exists = os.path.exists(model_path)

# Carrega modelo se já existir um checkpoint, caso contrário, o cria.
model = models.load_model(model_path) \
    if exists \
    else models.Sequential([
    layers.Resizing(56, 56),
    layers.Rescaling(1.0/255),
    layers.RandomRotation((-0.2, 0.2)),
    layers.Conv2D(32, (3, 3),
    activation = 'relu',
    kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3),
    activation = 'relu',
    kernel_initializer = initializers.RandomNormal()
    ),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128,
    activation = 'relu',
    kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(64,
    activation = 'relu',
    kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(64,
    activation = 'relu',
    kernel_initializer = initializers.RandomNormal()
    ),
    layers.Dense(1,
    activation = 'sigmoid',
    kernel_initializer = initializers.RandomNormal()
    )
])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
            learning_rate = learning_rate
        ),
        loss = losses.BinaryCrossentropy(),
        metrics = [ metrics.BinaryAccuracy(), metrics.CategoricalAccuracy() ]
    )


In [3]:
IMG_SIZE = 160
BATCH_SIZE = 32
FILE_PATH = r'C:\Users\eduar\OneDrive\Desktop\Repos\IAgro\project\dataset\Cotton leaves\40 Images'
CLASSES = os.listdir(FILE_PATH)
SEED = 123

print(CLASSES)

# Training dataset
train_ds = utils.image_dataset_from_directory(
    FILE_PATH,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    interpolation='nearest',
    class_names=CLASSES,
    shuffle=True
)

# Validation (test) dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    FILE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    label_mode='int',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    interpolation='nearest',
    class_names=CLASSES,
    shuffle=True
)


['Aphids', 'Army worm', 'Bacterial blight', 'Healthy', 'Powdery mildew', 'Target spot']
Found 4426 files belonging to 6 classes.
Using 3541 files for training.
Found 4426 files belonging to 6 classes.
Using 885 files for validation.


In [13]:
model.fit(train_ds,
    epochs = epochs,
    validation_data = val_ds,
    callbacks= [
        callbacks.EarlyStopping(
        monitor = 'val_loss',
        patience = patience,
        verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 16s 126ms/step - binary_accuracy: 0.1869 - categorical_accuracy: 0.2316 - loss: -1864018.1250 - val_binary_accuracy: 0.1774 - val_categorical_accuracy: 0.2893 - val_loss: -164453552.0000
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 15s 130ms/step - binary_accuracy: 0.1853 - categorical_accuracy: 0.1676 - loss: -2395086336.0000 - val_binary_accuracy: 0.1774 - val_categorical_accuracy: 0.2045 - val_loss: -44063002624.0000
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 15s 129ms/step - binary_accuracy: 0.1829 - categorical_accuracy: 0.1543 - loss: -133153390592.0000 - val_binary_accuracy: 0.1774 - val_categorical_accuracy: 0.1684 - val_loss: -940548030464.0000
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 15s 136ms/step - binary_accuracy: 0.1839 - categorical_accuracy: 0.1179 - loss: -1723094532096.0000 - val_binary_accuracy: 0.1774 - val_categorical_accuracy: 0.2169 - val_loss: -7210555932672.0000
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 15s 129ms/step - binary_